# Chat-GPT QA for Online Articles

In [1]:
#!pip install PyQT5

In [7]:
#Import required packages
import openai
from langchain.llms import AzureOpenAI
#This will help us create embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
#Using ChromaDB as a vector store for the embeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
import os

In [8]:
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [9]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.apple.com/newsroom/2023/06/ios-17-makes-iphone-more-personal-and-intuitive/")
data = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
splits = text_splitter.split_documents(loader.load())

In [11]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [12]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")

In [13]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [14]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

In [15]:
rag_chain.invoke("Tell me the new features in iOS 17").content

'The new features in iOS 17 include upgrades to the communications experience, easier sharing with AirDrop, more intelligent text input, and new experiences with Journal and StandBy. It also introduces Contact Posters, a new stickers experience, Live Voicemail, and more.'

In [16]:
rag_chain.invoke("What's new about messaging?").content

'New messaging features in iOS 17 include an expandable menu for iMessage apps, powerful search filters, a catch-up arrow to indicate where the user left off, and the ability to reply inline by swiping. Additionally, there are new stickers and the ability to create Live Stickers from photos, with the option to add effects and a new drawer for easier access.'

In [17]:
rag_chain.invoke("What are the new sharing features?").content

'The new sharing features in iOS 17 include AirDrop with new ways to share and NameDrop for easily sharing contact information by bringing iPhones or iPhones and Apple Watches together. Users can also share content or start SharePlay for listening to music, watching a movie, or playing a game in close proximity between iPhone devices.'